In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"kodipakaneha","key":"d3036fee819b5b6292a2128d54622348"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d piantic/plantpathology-apple-dataset

100% 811M/813M [00:15<00:00, 50.1MB/s]
100% 813M/813M [00:15<00:00, 56.5MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "plant-diseases.zip"

with ZipFile('plantpathology-apple-dataset.zip', 'r') as zipObj:
 
   zipObj.extractall("./")
print("done")

done


In [ ]:
import pandas as pd
df = pd.read_csv("./train.csv")
df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [ ]:
df.head(100)

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
95,Train_95,0,1,0,0
96,Train_96,0,0,1,0
97,Train_97,0,0,1,0
98,Train_98,0,0,1,0


In [ ]:

classes = ["healthy","multiple_dieseases","rust","scab"]


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in range(len(df)):
  df['image_id'][i]=i 
  
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,image_id,healthy,multiple_diseases,rust,scab
0,0,0,0,0,1
1,1,0,1,0,0
2,2,1,0,0,0
3,3,0,0,1,0
4,4,1,0,0,0


In [ ]:
!pip install tensorflow-gpu==1.13.1

In [ ]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   3867      0 --:--:-- --:--:-- --:--:--  3862
OK


In [ ]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   159k      0 --:--:-- --:--:-- --:--:--  159k
OK


In [ ]:
!pip install requests

In [ ]:
import os
import json
import random
import requests
import subprocess
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


%matplotlib inline
tf.__version__

'1.13.1'

In [ ]:

df.shape
train = df[0:3096]
test = df[3096:len(df)]


In [ ]:
print(train.shape,test.shape)

(3096, 5) (3096, 5)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train,test,test_size=0.3)

In [ ]:
x_train.shape

(2167, 5)

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(5,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=5, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
2167/2167 [==============================] - 0s 191us/sample - loss: 9113.1544 - acc: 0.8671
Epoch 2/5
2167/2167 [==============================] - 0s 96us/sample - loss: 18.2961 - acc: 1.0000
Epoch 3/5
2167/2167 [==============================] - 0s 95us/sample - loss: 18.2907 - acc: 1.0000
Epoch 4/5
2167/2167 [==============================] - 0s 93us/sample - loss: 17.9085 - acc: 1.0000
Epoch 5/5
2167/2167 [==============================] - 0s 92us/sample - loss: 17.8760 - acc: 1.0000


In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

929/929 [==============================] - 0s 192us/sample - loss: 26.0464 - acc: 1.0000
Test accuracy: 1.0


In [ ]:
model_name = 'jaimahishmati.h5'
tf.keras.models.save_model(model, model_name)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


In [ ]:
tflite_model = converter.convert()

In [ ]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)

In [ ]:
MODEL_DIR = "model/"
version = 1

In [ ]:
export_path = os.path.join(MODEL_DIR, str(version))
export_path

'model/1'

In [ ]:
if os.path.isdir(export_path):
    !rm -r {export_path}

In [ ]:
tf.saved_model.simple_save(tf.keras.backend.get_session(), export_dir=export_path, inputs={"input_image":model.input}, outputs={t.name:t for t in model.outputs})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: model/1/saved_model.pb


In [ ]:
os.environ['MODEL_DIR'] = os.path.abspath(MODEL_DIR)

In [ ]:
random_image = np.random.randint(0, len(x_test))
random_image

659

In [ ]:
%%bash --bg
nohup tensorflow_model_server --rest_api_port=8501 --model_name=cifar10 --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 2 in a separate thread.


In [ ]:
!apt-get update & apt-get install tensorflow-model-server

Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:8 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x8

In [ ]:
!tail server.log

nohup: failed to run command 'tensorflow_model_server': No such file or directory
